In [52]:
import pandas as pd  
import numpy as np
import re
import random
from unidecode import unidecode 

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep  

# Si no deja importar alguna de las librerías, probar a ejecutar el comando pertinente en la terminal:
#----- pip install pandas
#----- pip install numpy
#----- pip install regex
#----- pip install unidecode
#----- pip install selenium
#----- pip install webdriver-manager

In [48]:
def puntuacion_tomatoes (archivo_csv = 'Ruta o nombre del archivo.csv', columna1 = 'Nombre columna títulos', columna2 = 'Nombre columna años'):
    
    """
    Obtiene la puntuación de las películas en Rotten Tomatoes a partir de un archivo CSV de películas.

    Parámetros:
    archivo_csv (str): La ruta o nombre del archivo CSV que contiene los datos de las películas.
    columna1 (str): El nombre de la columna que contiene los títulos de las películas en el archivo CSV.
    columna2 (str): El nombre de la columna que contiene los años de las películas en el archivo CSV.

    Returns:
    pelis_con_puntuacion (lista): Una lista de tuplas que contiene el nombre de la película, el año y su puntuación en Rotten Tomatoes (o NaN si no se encuentra la puntuación o '-' si no hay puntuación disponible).

    """
    
    #! -------------------------------------------------------------------------------------- Apertura del archivo csv
    
    # Abre el archivo csv
    pelis_origen = pd.read_csv(f"{archivo_csv}")
    
    # Revisa cuál es el total de filas del archivo
    ultimo_elemento = pelis_origen.shape[0]
    
    # Crea una lista de tuplas que contiene el título de la película a buscar y su año
    pelis_buscar = [(pelis_origen.at[i, f'{columna1}'] , pelis_origen.at[i, f'{columna2}']) for i in range (0, ultimo_elemento)] # Se agrega a la lista una tupla con el nombre , año de la peli por cada índice dentro del rango 0 , ultimo elemento de pelis_drama_2000_2004

    #! -------------------------------------------------------------------------------------- Extracción de datos con Selenium

    driver = webdriver.Chrome()

    driver.get("https://www.google.com/")

    # Amplía la ventana a tamaño completo
    driver.maximize_window()

    sleep(3)

    ultimo_rango = len(pelis_buscar)

    # Rechaza las cookies en caso de haberlas
    try:
        driver.find_element("css selector" , "#W0wltc").click()
        
    except:
        pass

    pelis_con_puntuacion = []
    
    ultimo_descanso = -200
    
    print('--- Empieza la extracción de datos ---')
    print(' ')

    # Itera por rango, utilizando como delimitador el total de elementos de la lista de pelis a buscar
    for i in range (0 , ultimo_rango):
        
        # Condición para asegurar que una vez que se haya hecho un descanso, no se repita hasta pasado un mínimo de iteraciones
        if i >= ultimo_descanso + 200:
            descanso = random.randint(90, 210) # Escoge un número random y lo asigna a la variable descanso
            
            # Si el número de iteración es divisible por el número de descanso, se para durante una cantidad de segundos aleatoria
            if i != 0 and i % descanso == 0:
                tiempo_descanso = random.randint(30 , 330)
                ultimo_descanso = i 
                print(f"En la iteración número {i} me he ido a por un cafecito ☕. Volveré en {round(tiempo_descanso/60 , 1)} minutos")
                sleep(tiempo_descanso)
                print(f"¡Ya estoy de vuelta! Sigo con lo mío 💫")
                print(' ')
            
        nombre_peli = f'{pelis_buscar[i][0]}' # Saca el título de la película
        anio_peli = int(f'{pelis_buscar[i][1]}') # Saca el año de la película
        string_buscar = f"{nombre_peli}, {anio_peli} valoración rotten tomatoes" # Define la frase que se buscará. Ejemplo: Istota, 2000 valoración rotten tomatoes
        
        # Accede a la barra de búsqueda e introduce la frase
        driver.find_element("css selector", "#APjFqb").send_keys(string_buscar, Keys.ENTER)
        
        # Itera por los primeros 5 resultados de la primera página
        for resultado in range (1 , 5): 
            
            puntuacion = np.nan
            
            try:
                datos_resultado = driver.find_element('css selector', f'#rso > div:nth-child({resultado}) > div > div').text.split("\n") # Obtiene los datos del resultado de búsqueda según rango
                
                # Condición para verificar si en el segundo elemento de los resultados de búsqueda aparece 'rotten tomatoes' (así sabemos que la información está sacada de su página)
                if datos_resultado[1].lower() == 'rotten tomatoes':
                    
                    titulo = datos_resultado[0] # Asigna el nombre de la película según el resultado del buscador a la variable título
                    
                    # Condición para comprobar si el título del buscador coincide con el nombre de la película que estamos buscando
                    if unidecode(titulo).lower().strip() == unidecode(nombre_peli).lower().strip():
                        
                        # Si los nombres coinciden, itera por los elementos del resultado para encontrar la valoración
                        for elemento in datos_resultado:
                            
                            if 'valoracion' in unidecode(elemento).lower().strip(): 
                                valoracion = elemento
                                puntuacion = re.search('(\d+)%' , valoracion).group(1)
                                puntuacion = int(puntuacion.replace("%" , "")) # Si encuentra la valoración, asigna su valor a la variable
                                break 
                            
                            else:
                                puntuacion = "-" # Si no encuentra la valoración, le asigna '-' por defecto

                        break
                          
            except:
                pass
            
        sleep(random.randint(1, 4))           
        tupla = (nombre_peli , anio_peli , puntuacion) # Genera una tupla con el nombre de la película, el año y la puntuación que tiene en rotten tomatoes (valor, si la encuentra · nan, si no la encuentra · '-', si la encuentra, pero no tiene puntuación)
        pelis_con_puntuacion.append(tupla) # Añade la tupla a la lista
        
        # Limpia la barra de búsqueda
        driver.find_element("css selector" , "#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div").click()
    
    print('--- Extracción de datos completada 🙌 ---')
    return pelis_con_puntuacion           

In [49]:
tomatoes_drama_2000_2004 = puntuacion_tomatoes('Drama/pelis_drama_2000_2004.csv' , '1' , '2')

# SI DA ERROR: 
# ------------ Si el archivo no se encuentra en la misma ubicación que este jupyter, tendremos que darle la ruta. P.e: en lugar de 'pelis_drama_2000_2004.csv' >>> 'Drama/pelis_drama_2000_2004.csv'
# ------------ Hay que sustituir '1' y '2' por el nombre de las columnas en las que aparezca el título de la película y el año
# ------------ Por ejemplo: '1' >>> 'nombre' y '2' >>> 'aÃ±o estreno'
# ------------ Podemos saber el nombre de las columnas abriendo el archivo csv en un excel y mirando en la celda A1

[('Istota', 2000, nan), ('For the Cause', 2000, '-'), ('Florentino y el diablo', 2000, nan), ('Karobaar: The Business of Love', 2000, nan), ('Gang', 2000, '-'), ('Consequence', 2000, 12), ('Fa yeung nin wah', 2000, nan), ('Beyond the Pale', 2000, '-'), ('Chinese Coffee', 2000, 74), ('Passionnément', 2000, '-'), ('Love 101', 2000, 33), ('Pushing the Envelope', 2000, '-'), ('Way Past Cool', 2000, nan), ('State and Main', 2000, 85), ('Vulgar', 2000, 23), ('Bruno', 2000, 20), ('Lured Innocence', 2000, 0), ('Shikari', 2000, '-'), ('Capitães de Abril', 2000, nan), ('Sånger från andra våningen', 2000, nan), ('The Million Dollar Hotel', 2000, 24), ('Chicken Run', 2000, 97), ('The Highwayman', 2000, nan), ('Foiled', 2000, nan), ('Return to Me', 2000, 62), ('Waking the Dead', 2000, 50), ('Flamingo Dreams', 2000, nan), ('Dinosaur', 2000, 65), ('Duets', 2000, 21), ('Fallen Arches', 2000, '-'), ('Wind River', 2000, 57), ('Cement', 2000, 47), ('Waterproof', 2000, 60), ('Mothman', 2000, 22), ('The Ya

In [44]:
def guardar_csv (dict_json):
    
    df_dict_json = pd.DataFrame(dict_json)
    
    df_dict_json.to_csv(input("Indica el nombre o ruta del archivo. Recuerda poner .csv al final"))

In [51]:
guardar_csv(tomatoes_drama_2000_2004) # Ejemplo de input usando ruta del archivo: Drama/tomatoes_pelis_drama_2000_2004.csv